### 주가데이터를 수집하고, 그래프를 그려보고, 회귀분석까지 진행
  + reference: <파이썬을 활용한 금융공학 레시피>, 김용환
  + 네이버와 다음의 주가데이터 활용 (다음은 금융데이터 크롤링 X)

#### 네이버금융에서 KOSPI200 일별시세 자료로의 접근
  + KOSPI200 화면 --> '일별시세'에서 '프레임소스보기' --> 'view-source:' 부분을 제외한 url을 주소창에 입력
  + url구조 확인 (https://finance.naver.com/sise/sise_index_day.nhn?code=종목코드&page=페이지번호)

In [1]:
index_cd = 'KPI200'
page_n = 1
naver_index = \
'https://finance.naver.com/sise/sise_index_day.nhn?code=' + index_cd + \
'&page=' + str(page_n)

In [2]:
# urllib.request의 urlopen
from urllib.request import urlopen
source = urlopen(naver_index).read()
source

b'\n\n\n\n\n\n\n<html lang="ko">\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=euc-kr">\n<title>\xb3\xd7\xc0\xcc\xb9\xf6 \xb1\xdd\xc0\xb6</title>\n<link rel="stylesheet" type="text/css" href="/css/newstock.css?20201217181526">\n<link rel="stylesheet" type="text/css" href="/css/common.css?20201217181526">\n<link rel="stylesheet" type="text/css" href="/css/layout.css?20201217181526">\n<link rel="stylesheet" type="text/css" href="/css/main.css?20201217181526">\n<link rel="stylesheet" type="text/css" href="/css/newstock2.css?20201217181526">\n<link rel="stylesheet" type="text/css" href="/css/newstock3.css?20201217181526">\n<link rel="stylesheet" type="text/css" href="/css/world.css?20201217181526">\n</head>\n<body>\n<script type="text/javascript" src="/js/jindo.min.ns.1.5.3.euckr.js?20201217181526"></script>\n<script type="text/javascript" src="/js/lcslog.js?20201217181526"></script>\n\t\t\t\t<!-- \xc0\xcf\xba\xb0\xbd\xc3\xbc\xbc -->\n\t\t\t\t<div class="box_type_m">

In [3]:
# BeautifulSoup
from bs4 import BeautifulSoup
source = BeautifulSoup(source, 'lxml')

In [4]:
print(source.prettify())

<html lang="ko">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <title>
   네이버 금융
  </title>
  <link href="/css/newstock.css?20201217181526" rel="stylesheet" type="text/css"/>
  <link href="/css/common.css?20201217181526" rel="stylesheet" type="text/css"/>
  <link href="/css/layout.css?20201217181526" rel="stylesheet" type="text/css"/>
  <link href="/css/main.css?20201217181526" rel="stylesheet" type="text/css"/>
  <link href="/css/newstock2.css?20201217181526" rel="stylesheet" type="text/css"/>
  <link href="/css/newstock3.css?20201217181526" rel="stylesheet" type="text/css"/>
  <link href="/css/world.css?20201217181526" rel="stylesheet" type="text/css"/>
 </head>
 <body>
  <script src="/js/jindo.min.ns.1.5.3.euckr.js?20201217181526" type="text/javascript">
  </script>
  <script src="/js/lcslog.js?20201217181526" type="text/javascript">
  </script>
  <!-- 일별시세 -->
  <div class="box_type_m">
   <h4 class="top_tlt" style="text-align:left;">
    <em>
   

In [5]:
# 필요한 자료는 '날짜'와 '체결가(=종가지수)'
# 원하는 항목에 마우스 오른쪽 --> 검사
# 우리가 필요한 데이터는 모두 <td> 태그에 들어있음을 확인
td = source.find_all('td')
len(td)

54

In [6]:
# XPath (XML Path Language) 활용 --> 데이터를 대량으로 제공하는 사이트에서 원하는 데이터의 위치를 찾아낼 때 편리
# Python -> 숫자를 0부터 셈 / XPath -> 숫자를 1부터 셈 ==> 1을 빼줘야함
# /html/body/div/table[1]/tbody/tr[3]/td[1]
source.find_all('table')[0].find_all('tr')[2].find_all('td')[0]

<td class="date">2020.12.18</td>

In [7]:
d = source.find_all('td', class_='date')[0].text
d

'2020.12.18'

In [8]:
# datetime 라이브러리 활용
import datetime as dt
yyyy = int(d.split('.')[0])
mm = int(d.split('.')[1])
dd = int(d.split('.')[2])
this_date = dt.date(yyyy, mm, dd)
this_date

datetime.date(2020, 12, 18)

In [9]:
# 함수화
def date_format(d):
    d = str(d).replace('-','.') # 날짜 구분자가 '-'로 되어있을 때 '.'으로 변환
    yyyy = int(d.split('.')[0])
    mm = int(d.split('.')[1])
    dd = int(d.split('.')[2])
    
    this_date = dt.date(yyyy, mm, dd)
    return this_date

----

In [10]:
# 지수 가져오기
# XPath -> /html/body/div/table[1]/tbody/tr[3]/td[2]
this_close = source.find_all('tr')[2].find_all('td')[1].text
this_close = this_close.replace(',','') # 쉼표 제거
this_close = float(this_close)
this_close

371.09

In [11]:
p = source.find_all('td', class_ = 'number_1')[0].text
p

'371.09'

In [12]:
dates = source.find_all('td', class_='date')
prices = source.find_all('td', class_='number_1')

In [13]:
len(dates)

6

In [14]:
len(prices) # 체결가, 등략률, 거래량, 거래대금 => 4X6 ==> 첫 번째 값만 필요 (0,4,8...추출)

24

In [15]:
for n in range(len(dates)):
    this_date = dates[n].text
    this_date = date_format(this_date)
    
    this_close = prices[n*4].text # 0,4,8...
    this_close = this_close.replace(',','')
    this_close = float(this_close)
    this_close
    
    print(this_date, this_close)

2020-12-18 371.09
2020-12-17 372.0
2020-12-16 372.53
2020-12-15 370.88
2020-12-14 371.56
2020-12-11 372.24


In [16]:
# 페이지 내비게이션
# '맨뒤' 검사
paging = source.find('td', class_ = 'pgRR').find('a')['href']
paging

'/sise/sise_index_day.nhn?code=KPI200&page=617'

In [17]:
# 페이지 번호만 추출
paging = paging.split('&')[1]
paging

'page=617'

In [18]:
paging = paging.split('=')[1]
paging

'617'

In [19]:
# 페이지번호를 숫자 형식으로 변환
last_page = source.find('td', class_='pgRR').find('a')["href"]
last_page = last_page.split('&')[1]
last_page = last_page.split('=')[1]
last_page = int(last_page)

#### 데이터 추출 기능의 함수화

In [20]:
def historical_index_naver(index_cd, start_date='', end_date='', page_n=1, last_page=0):
    
    if start_date:   
        start_date = date_format(start_date)   
    else:    
        start_date = dt.date.today() 
    if end_date:   
        end_date = date_format(end_date)   
    else:   
        end_date = dt.date.today()  
        
        
    naver_index = \
    'http://finance.naver.com/sise/sise_index_day.nhn?code=' \
    + index_cd + '&page=' + str(page_n)
    
    # 지정한 페이지에서 코드 읽기
    source = urlopen(naver_index).read()
    # 뷰티풀 스프로 태그별로 코드 분류
    source = BeautifulSoup(source, 'lxml')   
    
    # <td class="date">태그에서 날짜 수집   
    dates = source.find_all('td', class_='date')  
    # <td class="number_1">태그에서 지수 수집
    prices = source.find_all('td', class_='number_1')  
    
    for n in range(len(dates)):
    
        if dates[n].text.split('.')[0].isdigit():
            
            # 날짜 처리
            this_date = dates[n].text
            this_date= date_format(this_date)
            
            if this_date <= end_date and this_date >= start_date:   
            # start_date와 end_date 사이에서 데이터 저장
                # 종가 처리
                this_close = prices[n*4].text   # 0,4,8...
                this_close = this_close.replace(',', '')
                this_close = float(this_close)

                # 딕셔너리에 저장
                historical_prices[this_date] = this_close
                
            elif this_date < start_date:   
            # start_date 이전이면 함수 종료
                return historical_prices              
            
    # 페이지 네비게이션
    if last_page == 0:
        last_page = source.find('td', class_='pgRR').find('a')['href']
        # 마지막페이지 주소 추출
        last_page = last_page.split('&')[1]
        last_page = last_page.split('=')[1]
        last_page = int(last_page) 
        
    # 다음 페이지 호출
    if page_n < last_page:   
        page_n = page_n + 1   
        historical_index_naver(index_cd, start_date, end_date, page_n, last_page)   
        
    return historical_prices

In [21]:
# 실행
# KOSPI200
index_cd = 'KPI200'
historical_prices = dict()

historical_index_naver(index_cd, '2020-12-01', '2020-12-17')
historical_prices

{datetime.date(2020, 12, 17): 372.0,
 datetime.date(2020, 12, 16): 372.53,
 datetime.date(2020, 12, 15): 370.88,
 datetime.date(2020, 12, 14): 371.56,
 datetime.date(2020, 12, 11): 372.24,
 datetime.date(2020, 12, 10): 369.37,
 datetime.date(2020, 12, 9): 371.47,
 datetime.date(2020, 12, 8): 363.45,
 datetime.date(2020, 12, 7): 369.86,
 datetime.date(2020, 12, 4): 367.63,
 datetime.date(2020, 12, 3): 361.66,
 datetime.date(2020, 12, 2): 358.66,
 datetime.date(2020, 12, 1): 352.4}